Connect to Drive

In [ ]:
from google.colab import drive
import os

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/llama3')

Mounted at /content/gdrive


Create Virtual Environment to run the project

In [ ]:
!pip install virtualenv
!virtualenv llama3-venv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 25.0 MB/s eta 0:00:00
created virtual environment CPython3.10.12.final.0-64 in 34407ms
  creator CPython3Posix(dest=/content/gdrive/My Drive/llama3/llama3-venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.2, setuptools==75.2.0, wheel==0.44.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [ ]:
import sys
!source llama3-venv/bin/activate
sys.path.append('/content/myenv/lib/python3.9/site-packages')

Install required packages

In [ ]:
!pip install --upgrade pip


!pip install accelerate==0.34.2 astunparse==1.6.3 attrs==24.2.0 bitsandbytes==0.43.3 blobfile==3.0.0 certifi==2024.8.30 charset-normalizer==3.3.2 cmake==3.30.2 colorama==0.4.6 exceptiongroup==1.2.2 expecttest==0.2.1 fairscale==0.4.13 filelock==3.13.1 fire==0.6.0 fsspec==2024.2.0 huggingface-hub==0.24.6 hypothesis==6.112.0 idna==3.8 jinja2==3.1.3 lintrunner==0.12.5 lxml==5.3.0 markupsafe==2.1.5 mpmath==1.3.0 networkx==3.2.1 ninja==1.11.1.1 numpy==1.26.3 optree==0.12.1 packaging==24.1 pillow==10.2.0 psutil==6.0.0 pycryptodomex==3.20.0 pyyaml==6.0.2 regex==2024.7.24 requests==2.32.3 safetensors==0.4.5 six==1.16.0 sortedcontainers==2.4.0 sympy==1.13.1 termcolor==2.4.0 tiktoken==0.4.0 tokenizers==0.19.1 torch==2.4.1+cu118 torchaudio==2.4.1+cu118 torchvision==0.19.1+cu118 tqdm==4.66.5 transformers==4.44.2 types-dataclasses==0.6.6 typing-extensions==4.9.0 urllib3==2.2.2
#!pip install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1
!pip install torch==2.5.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install xformers --index-url https://download.pytorch.org/whl/cu121
!pip install nltk numpy scikit-learn triton
!pip install python-docx PyPDF2 pandas datasets


# !pip install --no-deps xformers trl peft accelerate bitsandbytes
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl.metadata (5.3 kB)
ERROR: Could not find a version that satisfies the requirement torch==2.4.1+cu118 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0)
ERROR: No matching distribution found for torch==2.4.1+cu118
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

#Data Preprocess

In [ ]:
import io
import csv
import re
import docx
import PyPDF2
from google.colab import files
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer


nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('stopwords', quiet=True)

def extract_text_from_txt(file_content):
    return file_content.decode('utf-8')

def extract_text_from_docx(file_content):
    doc = docx.Document(io.BytesIO(file_content))
    return ' '.join([paragraph.text for paragraph in doc.paragraphs])

def extract_text_from_pdf(file_content):
    pdf_reader = PyPDF2.PdfReader(io.BytesIO(file_content))
    return ' '.join([page.extract_text() for page in pdf_reader.pages])

def extract_text_from_file(file_content, file_name):
    file_extension = file_name.split('.')[-1].lower()

    if file_extension == 'txt':
        return extract_text_from_txt(file_content)
    elif file_extension in ['doc', 'docx']:
        return extract_text_from_docx(file_content)
    elif file_extension == 'pdf':
        return extract_text_from_pdf(file_content)
    else:
        raise ValueError(f"Unsupported file format: {file_extension}")

def count_syllables(word):
    vowels = 'aeiou'
    word = word.lower()
    count = 0
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith('e'):
        count -= 1
    if count == 0:
        count += 1
    return count

def analyze_writing_style(text):
    if len(text.split()) < 1000:
        return "Error: The input text should have at least 1000 words."

    words = word_tokenize(text.lower())
    sentences = sent_tokenize(text)

    word_count = len(words)
    sentence_count = len(sentences)
    avg_sentence_length = word_count / sentence_count

    unique_words = set(words)
    vocabulary_richness = len(unique_words) / word_count

    pos_tags = pos_tag(words)
    desired_pos = {'JJ', 'JJR', 'JJS',  # Adjectives
                   'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ',  # Verbs
                   'NN', 'NNS', 'NNP', 'NNPS',  # Nouns
                   'RB', 'RBR', 'RBS'}  # Adverbs
    filtered_words = [word.lower() for word, pos in pos_tags if pos in desired_pos]
    word_freq = Counter(filtered_words)
    most_common_words = word_freq.most_common(30)

    pos_counts = Counter(tag for word, tag in pos_tags)

    complex_sentence_count = sum(1 for sentence in sentences if len(word_tokenize(sentence)) > 20)
    complex_sentence_ratio = complex_sentence_count / sentence_count

    total_syllables = sum(count_syllables(word) for word in words)
    flesch_kincaid = 0.39 * (word_count / sentence_count) + 11.8 * (total_syllables / word_count) - 15.59

    vectorizer = TfidfVectorizer(stop_words='english', max_features=10)
    tfidf_matrix = vectorizer.fit_transform([text])
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = tfidf_matrix.toarray()[0]
    distinctive_words = [feature_names[i] for i in tfidf_scores.argsort()[::-1][:10]]

    summary = f"""Follow this Writing Style Summary while writing the letter:1. Vocabulary: - {vocabulary_richness:.2f} - commonly use these adjectives, verbs, nouns, or adverbs: {', '.join(f'{word}' for word, count in most_common_words)} ,2. Sentence Structure: - Follow the sentence length: {avg_sentence_length:.2f} words - Follow the Complex sentence ratio:{complex_sentence_ratio:.2f} , 3. Parts of Speech:- Nouns: {pos_counts.get('NN', 0) + pos_counts.get('NNS', 0) + pos_counts.get('NNP', 0) + pos_counts.get('NNPS', 0)} ({(pos_counts.get('NN', 0) + pos_counts.get('NNS', 0) + pos_counts.get('NNP', 0) + pos_counts.get('NNPS', 0)) / word_count:.2%}) - Verbs: {sum(pos_counts.get(tag, 0) for tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'])} ({sum(pos_counts.get(tag, 0) for tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']) / word_count:.2%}) - Adjectives: {pos_counts.get('JJ', 0) + pos_counts.get('JJR', 0) + pos_counts.get('JJS', 0)} ({(pos_counts.get('JJ', 0) + pos_counts.get('JJR', 0) + pos_counts.get('JJS', 0)) / word_count:.2%}) - Adverbs: {pos_counts.get('RB', 0) + pos_counts.get('RBR', 0) + pos_counts.get('RBS', 0)} ({(pos_counts.get('RB', 0) + pos_counts.get('RBR', 0) + pos_counts.get('RBS', 0)) / word_count:.2%}) , 4. follow this Readability:- Flesch-Kincaid Grade Level: {flesch_kincaid:.2f}, 5. Follow this Style Characteristics: - {'Uses complex sentences frequently' if complex_sentence_ratio > 0.3 else 'Prefers simpler sentence structures'}- {'Rich and diverse vocabulary' if vocabulary_richness > 0.2 else 'More focused vocabulary'} - {'Formal tone' if flesch_kincaid > 10 else 'Casual tone'}"""
    return summary

def create_csv_from_text(input_text, output_file):
    # Split the text into sentences
    sentences = re.split(r'(?<=[.!?])\s+', input_text.strip())

    # Remove the last sentence if there's an odd number of sentences
    if len(sentences) % 2 != 0:
        sentences = sentences[:-1]

    # Prepare the data for CSV
    csv_data = []
    for i in range(0, len(sentences), 2):
        row = [

                {'role': 'system', 'content': 'Mimick the writing style'},
                {'role': 'user', 'content': 'Follow the writing style and predict the next sentence for the given line: '+ sentences[i]},
                {'role': 'assistant', 'content': sentences[i+1]}

        ]
        csv_data.append(row)

    # Write to CSV file
    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['conversations'])  # Header
        for row in csv_data:
            writer.writerow([str(row)])


# Main execution
print("Please upload a document file (TXT, DOC, DOCX, or PDF) which consists atleast 3000 words of your writing sample:")
uploaded = files.upload()

for filename, file_content in uploaded.items():
    try:
        extracted_text = extract_text_from_file(file_content, filename)

        if len(extracted_text.split()) < 1000:
            print(f"Error: The extracted text from {filename} should have at least 1000 words.")
        else:
            result = analyze_writing_style(extracted_text)
            print(f"Analysis for {filename}:")
            #print(result)
    except Exception as e:
        print(f"An error occurred while processing {filename}: {str(e)}")

summary = result


for filename, file_content in uploaded.items():
    try:
        extracted_text = extract_text_from_file(file_content, filename)

        if len(extracted_text.split()) < 1000:
            print(f"Error: The extracted text from {filename} should have at least 1000 words.")
        else:
            output_file = '/content/gdrive/My Drive/llama3/output.csv'
            create_csv_from_text(extracted_text, output_file)
            print(f"CSV file '{output_file}' has been created successfully.")
            # files.download(output_file)
    except Exception as e:
        print(f"An error occurred while processing {filename}: {str(e)}")


Please upload a document file (TXT, DOC, DOCX, or PDF) which consists atleast 3000 words of your writing sample:


Saving My writing sample.docx to My writing sample (6).docx
Analysis for My writing sample (6).docx:
CSV file '/content/gdrive/My Drive/llama3/output.csv' has been created successfully.


Convert the CSV data

In [ ]:
import pandas as pd
from datasets import Dataset
from unsloth.chat_templates import get_chat_template
import ast
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)


# Load your CSV data
csv_data = pd.read_csv("/content/gdrive/My Drive/llama3/output.csv")

# Convert the string representation of list of dictionaries to actual list of dictionaries
csv_data['conversations'] = csv_data['conversations'].apply(ast.literal_eval)

# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(csv_data)

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

print(dataset)
dataset = dataset.map(formatting_prompts_func, batched=True)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.3: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers via:
`pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"`


Dataset({
    features: ['conversations'],
    num_rows: 76
})


Map:   0%|          | 0/76 [00:00<?, ? examples/s]

Save the dataset

In [ ]:
# Save the processed dataset to disk
output_directory = "/content/gdrive/My Drive/llama3/preprocessed_data"
dataset.save_to_disk(output_directory)

print(f"Dataset saved to {output_directory}")

# Optional: If you want to download the dataset to your local machine from Google Colab
from google.colab import files
import shutil

# Compress the dataset directory
shutil.make_archive("/content/preprocessed_data", 'zip', output_directory)

# Download the compressed dataset
files.download("/content/preprocessed_data.zip")

Saving the dataset (0/1 shards):   0%|          | 0/76 [00:00<?, ? examples/s]

Dataset saved to /content/gdrive/My Drive/llama3/preprocessed_data


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Finetune Data

In [ ]:
import json
import torch
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel
from datasets import load_from_disk

In [ ]:
# Loading the model and the tokinizer for the model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)

# Setup for QLoRA/LoRA peft of the base model
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 42,
    use_rslora = True,
    use_dora = False,
    loftq_config = None,
    #task_type = "CAUSAL_LM",  # Added task type
    fan_in_fan_out = False,  # Added fan_in_fan_out parameter
    init_lora_weights =True,
)



==((====))==  Unsloth 2024.10.3: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.10.3 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
# Define a function to format the conversations
def format_conversations(example):
    formatted_text = ""
    for message in example['conversations']:
        if message['role'] == 'system':
            formatted_text += f"<|start_header_id|>system<|end_header_id|>{message['content']}<|eot_id|>"
        elif message['role'] == 'user':
            formatted_text += f"<|start_header_id|>user<|end_header_id|>{message['content']}<|eot_id|>"
        elif message['role'] == 'assistant':
            formatted_text += f"<|start_header_id|>assistant<|end_header_id|>{message['content']}<|eot_id|>"
    return {"formatted_text": formatted_text}

# Apply the formatting function to the dataset
dataset_train = load_from_disk("/content/gdrive/My Drive/llama3/preprocessed_data")
dataset_train = dataset_train.map(format_conversations)

# Setting up the trainer for the model
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    dataset_text_field = "formatted_text",  # Use the new formatted field
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        warmup_steps = 10,
        max_steps = 0,
        num_train_epochs= 1,
        learning_rate = 2e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type =  "cosine",
        seed = 42,
        output_dir = "outputs",
        max_grad_norm = 1.0,  # Added gradient clipping
        save_steps = 50,  # Added to save checkpoints
        #evaluation_strategy = "steps",  # Added for periodic evaluation
        eval_steps = 50,  # Evaluate every 50 steps
    ),
)



Map:   0%|          | 0/76 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/76 [00:00<?, ? examples/s]

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Training the model
wandb.init()
trained_model = trainer.train()

train/epoch,▁▃▆██
train/global_step,▁▃▆██
train/grad_norm,▁ █▄
train/learning_rate,▁▁▄█
train/loss,▁▆█▇
total_flos,258080276054016.0
train/epoch,0.84211
train/global_step,4
train/grad_norm,40.86747
train/learning_rate,1e-05
train/loss,5.6302


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers and Unsloth!


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 76 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 4
 "-____-"     Number of trainable parameters = 83,886,080


In [ ]:
with open("trainer_stats.json", "w") as f:
    json.dump(trained_model, f, indent=4)

model.save_pretrained("llama-3-8b-Instruct-mimick")

#Run the inference

In [ ]:
# Loading the fine-tuned model and the tokenizer for inference
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "llama-3-8b-Instruct-mimick",
        max_seq_length = 2048,
        dtype = torch.float16,
        load_in_4bit = True,
    )

# Using FastLanguageModel for fast inference
FastLanguageModel.for_inference(model)
prompt = input("Enter the prompt to write Letter, Email, messages, Essay in your style")
# Tokenizing the input and generating the output
print("\n\n")
inputs = tokenizer(
[
    "<|start_header_id|>system<|end_header_id|>"+summary+"<|eot_id|><|start_header_id|>user<|end_header_id|> "+prompt+"<|eot_id|>"
], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs, skip_special_tokens = True)



==((====))==  Unsloth 2024.10.3: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Enter the prompt to write Letter, Email, messages, Essay in your stylewrite a leave letter to manager saying I have typhoid fever and could not work for next 1 week.





["systemFollow this Writing Style Summary while writing the letter:1. Vocabulary: - 0.29 - commonly use these adjectives, verbs, nouns, or adverbs: i, work, am, was, computer, is, research, science, project, team, analysis, application, are, have, role, experience, also, mobile, learning, working, new, time, more, knowledge, projects, learn, engineering, machine, company, skills,2. Sentence Structure: - Follow the sentence length: 21.73 words - Follow the Complex sentence ratio:0.49, 3. Parts of Speech:- Nouns: 1002 (29.94%) - Verbs: 498 (14.88%) - Adjectives: 301 (8.99%) - Adverbs: 89 (2.66%), 4. follow this Readability:- Flesch-Kincaid Grade Level: 11.89, 5. Follow this Style Characteristics: - Uses complex sentences frequently- Rich and diverse vocabulary - Formal toneuser write a leave letter to manager saying I have typhoid fever and could not work for next 1 week.assistant\n\nDear [Manager's Name],\n\nI am writing to inform you that, unfortunately, I have been diagnosed with typh

In [ ]:
a=tokenizer.batch_decode(outputs)[0].split("<|end_header_id|>")[len(tokenizer.batch_decode(outputs)[0].split("<|end_header_id|>"))-1 ]
print(a[:len(a)-10])



Dear [Manager's Name],

I am writing to inform you that, unfortunately, I have been diagnosed with typhoid fever and will be unable to work for the next week. As you can understand, my health and well-being are my top priority, and I must take the necessary time to recover and receive proper medical treatment.

I am currently undergoing treatment and am expected to make a full recovery, but I will require some time to rest and recuperate. I apologize for any inconvenience this may cause and will do my best to catch up on any missed work as soon as I am feeling better.

I appreciate your understanding and support during this time. If there is anything I can do to mitigate the impact of my absence, please do not hesitate to reach out to me. I will be available by email or phone if you need to contact me.

Thank you for your attention to this matter, and I look forward to returning to work as soon as possible.

Sincerely,
[Your Name]
